In [2]:
import cv2
import os
import pickle
import os.path as path
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_path = '/content/drive/MyDrive/Face Recognition using ML/NewDataset'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories,labels))

In [6]:
label_dict

{'Alec_Baldwin': 9,
 'Annie_Ilonzeh': 10,
 'Chris_Evans': 11,
 'Chris_Rock': 8,
 'Danny_Glover': 12,
 'Kristen_Johnston': 6,
 'Lisa_Kudrow': 2,
 'Melissa_Fumero': 7,
 'Oliver_Platt': 4,
 'Peri_Gilpin': 1,
 'Robert_Downey_Jr': 0,
 'Ryan_Reynolds': 3,
 'Seth_Rogen': 5}

In [7]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

In [8]:
def extract_face(img,face_cascade,img_size):
    lis = []
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img) 
    if(len(face_rects) == 1):
        for (x,y,w,h) in face_rects:
            i2 = face_img[y:y+h,x:x+w]
            resized = cv2.resize(i2,(img_size,img_size))
            lis.append(resized)
    return lis

In [9]:
img_size = 300
data = []
target = []

train_datagen = ImageDataGenerator()#shear_range=0.2,zoom_range=0.2,horizontal_flip=True,rotation_range=25)#horizontal_flip=True

count = 1

fnames = []

for category in categories:
    folder_path = os.path.join(data_path,category)
    folder_path = os.path.join(folder_path,'face')
    img_names = os.listdir(folder_path)
        
    for img_name in img_names:
        filename = os.path.join(folder_path,img_name)
        fnames.append(filename)
        img = cv2.imread(filename)
        #print(img.shape)
        try:
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)   
            resized = cv2.resize(gray,(img_size,img_size)) #comment when training only faces
#             lis=extract_face(gray,face_cascade,img_size)#resized->gray
#             if(len(lis)==0):
#                 continue
#             resized=lis[0]
            new_img = expand_dims(resized, 0)
            train_set=train_datagen.flow(new_img, batch_size=1)
            for i in range(3):
                #plt.subplot(110 + 1 + i)
                batch=train_set.next()
                image=batch[0].astype('uint8')
                #print(image)
                #print("$"*30)
                augmented_gray_scale=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
                #print(augmented_gray_scale.shape)---(300,300)
                #fig=plt.figure(figsize=(12,16))
                #plt.imshow(augmented_gray_scale,cmap='gray')
                #plt.show()
                if image is not None:
                    # print(count)
                    data.append(augmented_gray_scale)
                    target.append(label_dict[category])
                    count=count + 1
            #plt.show()

        except Exception as e:
            print('Exception:',e)
print(count - 1)

KeyboardInterrupt: ignored

In [9]:
 np.array(data).shape

In [10]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
#new_target=np_utils.to_categorical(target)

In [11]:

np.save('/content/drive/MyDrive/Face Recognition using ML/bigdata_last_submission.npy',data)
np.save('/content/drive/MyDrive/Face Recognition using ML/bigtarget_last_submisison.npy',target)
# data_last_submission
# target_last_submisison
# working fine

In [12]:
len(data)

3240

In [13]:
data.shape

(3240, 300, 300, 1)

In [14]:
print(target)

[ 0  0  0 ... 12 12 12]


In [15]:
f=open('/content/drive/MyDrive/Face Recognition using ML/latest_store.pckl', 'wb')
new_dict=dict([(value, key) for key, value in label_dict.items()]) 
pickle.dump(new_dict, f)
f.close()

In [16]:
f=open('/content/drive/MyDrive/Face Recognition using ML/latest_target.pckl', 'wb')
pickle.dump(target, f)
f.close()

In [17]:
f=open('/content/drive/MyDrive/Face Recognition using ML/latest_filenames.pckl', 'wb')
pickle.dump(fnames, f)
f.close()